In [16]:
import pandas as pd
from difflib import get_close_matches
from datetime import datetime  # Add this import

df = pd.read_csv('/content/drive/MyDrive/Google_Hackathon/updated_dataset.csv')

def closest_match(symptom, symptoms_list):
    closest = get_close_matches(symptom, symptoms_list, n=1, cutoff=0.8)  # Adjust the cutoff threshold as needed
    return closest[0] if closest else None

def parse_schedule(schedule_str):
    # Parse the schedule string to extract start and end times
    start, end = schedule_str.split('-')
    start_time = datetime.strptime(start.strip(), "%I:%M %p")
    end_time = datetime.strptime(end.strip(), "%I:%M %p")
    return start_time, end_time

def recommend(symptom_1, symptom_2, symptom_3):
    if symptom_1 == symptom_2 or symptom_1 == symptom_3 or symptom_2 == symptom_3:
        return "Symptoms 1, 2, and 3 should be different and unique."

    all_symptoms = set([symptom_1, symptom_2, symptom_3])
    all_recommendations = {}

    for symptom in all_symptoms:
        closest_symptom = closest_match(symptom, df[['Symptom_1', 'Symptom_2', 'Symptom_3']].values.flatten().tolist())

        if closest_symptom:
            # filter the dataset and match closest
            filtered_df = df[df.apply(lambda x: closest_symptom in x.values, axis=1)]

            if filtered_df.empty:
                continue

            # Get recommendations for each disease associated with the symptom
            for idx, row in filtered_df.iterrows():
                disease = row['Disease']
                recommendation = {
                    "Name": row['Name'],
                    "City": row['City'],
                    "State": row['State'],
                    "Rating": row['user_rating'],
                    "Schedule": parse_schedule(row['schedule'])
                }
                if disease in all_recommendations:
                    all_recommendations[disease].append(recommendation)
                else:
                    all_recommendations[disease] = [recommendation]

    # If no recommendations are found based on any symptom, return an appropriate message
    if not all_recommendations:
        return "No recommendation found"

    # Sort recommendations based on schedule availability
    closest_time = None
    selected_doctors = []
    for disease, details in all_recommendations.items():
        for detail in details:
            if closest_time is None or detail['Schedule'][0] < closest_time:
                closest_time = detail['Schedule'][0]
                selected_doctors = [detail]
            elif detail['Schedule'][0] == closest_time:
                selected_doctors.append(detail)

    # If multiple doctors with the same specialization and closest free time, prioritize by rating
    if len(selected_doctors) > 1:
        selected_doctors.sort(key=lambda x: x['Rating'], reverse=True)

    return {disease: selected_doctors[:1] for disease in all_recommendations.keys()}

# Input symptoms
symptom_1 = input("Enter symptom 1: ")
symptom_2 = input("Enter symptom 2: ")
symptom_3 = input("Enter symptom 3: ")

# Get recommendations
recommendations = recommend(symptom_1, symptom_2, symptom_3)
print("Recommendations:")
if isinstance(recommendations, str):
    print(recommendations)
else:
    for disease, details in recommendations.items():
        print("Disease:", disease)
        for detail in details:
            print("Name:", detail["Name"])
            print("City:", detail["City"])
            print("State:", detail["State"])
            print("Rating:", detail["Rating"])
            print("Schedule:", detail["Schedule"][0].strftime("%I:%M %p"), "-", detail["Schedule"][1].strftime("%I:%M %p"))
            print()


Enter symptom 1: joint_pain
Enter symptom 2: skin_rash
Enter symptom 3: skin_peeling
Recommendations:
Disease: Psoriasis
Name: DR. PANDEY S.S.
City: VARANASI
State: UTTAR PRADESH
Rating: 5
Schedule: 09:00 AM - 09:30 AM

Disease: Fungal infection
Name: DR. PANDEY S.S.
City: VARANASI
State: UTTAR PRADESH
Rating: 5
Schedule: 09:00 AM - 09:30 AM

Disease: Acne
Name: DR. PANDEY S.S.
City: VARANASI
State: UTTAR PRADESH
Rating: 5
Schedule: 09:00 AM - 09:30 AM

Disease: Impetigo
Name: DR. PANDEY S.S.
City: VARANASI
State: UTTAR PRADESH
Rating: 5
Schedule: 09:00 AM - 09:30 AM

